In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
from torch.autograd import Variable

import glob
import os
import matplotlib.pyplot as plt
import torchvision.models as models
import cv2
import pickle
import random

In [2]:
class Classifier(nn.Module):
    def __init__(self, input_size, hidden_layers, output_size):
        super(Classifier, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_layers)
        self.fc2 = nn.Linear(hidden_layers, hidden_layers)
        self.fc3 = nn.Linear(hidden_layers, output_size)
    
    def forward(self,data):
        x = F.tanh(self.fc1(data))
        x = F.tanh(self.fc2(x))
        x = self.fc3(x)
        return F.softmax(x,dim = 1)
        

In [3]:
def label2onehot(label_id):
    vector = np.zeros(101)
    vector[label_id] = 1
    return vector

def onehot2label(vector):
    return np.argmax(vector)

In [4]:
def chunk(data,batch_size):
    batches = []
    x = np.arange(len(data) // batch_size + 2)*batch_size
#     print(x)
    for b,e in zip(x[0:-1],x[1:]):
        batches.append(data[b:e])
    return batches    

In [5]:
def encode_batch(minibatch):
    label_batch = []
    feature_batch_vector = []
    for label, feature in minibatch:
        label_batch.append(label)
        feature_batch_vector.append(feature.squeeze())
    return np.array(label_batch), np.array(feature_batch_vector)
    

In [6]:
def validate(test_data,net):
    correct = 0
    for label, feature in test_data:
        x = Variable(torch.FloatTensor(feature)).cuda()
        output = net(x)
        if onehot2label(output.cpu().data.numpy()) == label:
            correct += 1
    return correct / len(test_data)
        
        

In [7]:
with open("training_data_vgg19.pkl","rb") as f:
    training_data = pickle.load(f)
    random.shuffle(training_data)
    
with open("test_data_vgg19.pkl","rb") as f:
    test_data = pickle.load(f)

In [8]:
training_batches = chunk(training_data,128)

In [ ]:
net = Classifier(4096,2048,101)
learning_rate = 0.0001
optimizer = optim.Adam(net.parameters(), lr=learning_rate )
loss = nn.CrossEntropyLoss()
net.train()
net.cuda()
max_epoch = 150
test_acc = []
train_acc = []
test_loss = []
train_loss = []
for ep in range(max_epoch):
    err_sum = 0 
    for minibatch in training_batches:
        feature_label, feature_vector = encode_batch(minibatch)
        feature_tensor = Variable(torch.FloatTensor(feature_vector)).cuda()
        feature_label = Variable(torch.LongTensor(feature_label)).cuda()
        output = net(feature_tensor)
        error = loss(output, feature_label)
        err_sum += error
        optimizer.zero_grad()
        error.backward()
        optimizer.step()
    avg_err = err_sum.cpu().data.numpy()/ len(minibatch)
    test_acc.append(validate(test_data,net))
    train_acc.append(validate(training_data,net))
    print("ep[%d]\t training acc: %f\t test acc: %f"%(ep, train_acc[-1], test_acc[-1]))

ep[0]	 training acc: 0.330799	 test acc: 0.338178
ep[1]	 training acc: 0.377282	 test acc: 0.372040
ep[2]	 training acc: 0.402419	 test acc: 0.384990
ep[3]	 training acc: 0.422525	 test acc: 0.392079
ep[4]	 training acc: 0.441773	 test acc: 0.399485
ep[5]	 training acc: 0.454869	 test acc: 0.402772
ep[6]	 training acc: 0.464956	 test acc: 0.404238
ep[7]	 training acc: 0.473405	 test acc: 0.405228
ep[8]	 training acc: 0.480547	 test acc: 0.407564
ep[9]	 training acc: 0.487254	 test acc: 0.407802
ep[10]	 training acc: 0.493353	 test acc: 0.409307
ep[11]	 training acc: 0.499822	 test acc: 0.409743
ep[12]	 training acc: 0.504561	 test acc: 0.410772
ep[13]	 training acc: 0.512614	 test acc: 0.411406
ep[14]	 training acc: 0.522780	 test acc: 0.418416
ep[15]	 training acc: 0.527796	 test acc: 0.418733
ep[16]	 training acc: 0.536721	 test acc: 0.421149
ep[17]	 training acc: 0.541619	 test acc: 0.424277
ep[18]	 training acc: 0.549804	 test acc: 0.425267
ep[19]	 training acc: 0.554675	 test acc:

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
plt.plot(np.array(train_acc))
plt.plot(np.array(test_acc))
plt.grid()
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend(["Train","Test"])
plt.show()

In [ ]:
len(training_data)  + len(test_data) 